# Correlating full frame calcium imaging between somatic and dendritic ROIs

* 10/31/2024 - setup

## conclusion: there is pretty high correlation between somatic and dendritic frames in this dataset during locomotion and rest. 

In [29]:
from os.path import join
import json
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd

In [24]:
sima_dir = "/data2/gergely/invivo_DATA/sleep/dock11c5/8_2/TSeries-08022024-1036-001/TSeries-08022024-1036-001.sima/"
ff_ca_data = pd.read_csv(join(sima_dir, "s2p_ff", "ff_ca_data.csv"))
mob_immobs = []

data = join(sima_dir, "behavior", "mobility_immobility.json")
with open(data, "r") as f:
    mob_immobs = np.array(json.load(f))

In [26]:
len(mob_immobs)

20374

In [9]:
# Pivot the DataFrame to wide format, aligning values based on 'Frame' for each 'Plane'
wide_df = ff_ca_data.pivot(index="Frame", columns="Plane", values="AveragePixelValue")

# Drop any rows with NaN values (if there's a mismatch in frames)
wide_df = wide_df.dropna()

# Calculate the correlation between 'plane0' and 'plane1'
correlation = wide_df["plane0"].corr(wide_df["plane1"])

print(f"Correlation between 'plane0' and 'plane1': {correlation}")


Correlation between 'plane0' and 'plane1': 0.8771130182874436


In [33]:
# Verify the length of JSON data matches the number of rows in wide_df
if len(mob_immobs) != len(wide_df):
    # Interpolate the JSON data to match the length of wide_df
    x_original = np.linspace(0, 1, len(mob_immobs))
    x_target = np.linspace(0, 1, len(wide_df))
    interpolator = interp1d(x_original, mob_immobs, kind="linear")
    mob_immobs = interpolator(x_target)

# Convert the interpolated values to integers
mob_immobs = mob_immobs.astype(int)

# Add the JSON data (interpolated if needed) as a new column to wide_df
wide_df["mobility_immobility"] = mob_immobs

# Filter plane0 and plane1 data based on mobility_immobility values
plane0_mobility_0 = wide_df[wide_df["mobility_immobility"] == 0]["plane0"]
plane1_mobility_0 = wide_df[wide_df["mobility_immobility"] == 0]["plane1"]
plane0_mobility_1 = wide_df[wide_df["mobility_immobility"] == 1]["plane0"]
plane1_mobility_1 = wide_df[wide_df["mobility_immobility"] == 1]["plane1"]

# Calculate correlations
correlation_mobility_0 = plane0_mobility_0.corr(plane1_mobility_0)
correlation_mobility_1 = plane0_mobility_1.corr(plane1_mobility_1)

print(
    f"Correlation between plane0 and plane1 for mobility = 0: {correlation_mobility_0}"
)
print(
    f"Correlation between plane0 and plane1 for mobility = 1: {correlation_mobility_1}"
)


Correlation between plane0 and plane1 for mobility = 0: 0.8824605691656683
Correlation between plane0 and plane1 for mobility = 1: 0.8676099144669701
